# Frequency/Amplitude Study of the Epileptor 6D

- 6/12/18: from Marisa, I learned how the Epileptor fast system has regions of varying bifurcations, which produce different amplitude/frequency characteristics of the limit cycle (i.e. seizure period). Here, we want to produce simulations and track the trajectory of the fast subsystem variables

In [ ]:
import sys
import numpy as np
import os

sys.path.append('../../')
from tvb.simulator.lab import *
import tvbsim

# to run simulation and post processing and data loading
from tvbsim.postprocess.postprocess import PostProcessor
from tvbsim.postprocess.detectonsetoffset import DetectShift
from tvbsim.maintvbexp import MainTVBSim
from tvbsim.io.patient.subject import Subject
from tvbsim.base.constants.config import Config

# to run plotting at the end
from tvbsim.visualize.plotter_sim import PlotterSim
from tvbsim.base.dataobjects.timeseries import TimeseriesDimensions, Timeseries 
from collections import OrderedDict

2018-06-12 18:54:17,229 - WARNING - tvb.simulator.common - psutil module not available: no warnings will be issued when a
    simulation may require more memory than available
   INFO  log level set to INFO


In [ ]:

def save_processed_data(filename, times, epits, seegts, zts, state_vars):
    print('finished simulating!')
    print(epits.shape)
    print(seegts.shape)
    print(times.shape)
    print(zts.shape)
    print(state_vars.keys())

    # save tseries
    np.savez_compressed(filename, epits=epits, 
                                seegts=seegts,
                                times=times, 
                                zts=zts, 
                                state_vars=state_vars)
    
def process_weights(conn, shuffle=False, patient=None, other_pats=[]):
    if shuffle:
        if other_pats and patient is not None:
            # shuffle across patients
            randpat = MainTVBSim().randshufflepats(other_pats, patient)   
            shuffled_connfile = os.path.join(metadatadir, randpat, 'tvb', 'connectivity.zip')
            if not os.path.exists(shuffled_connfile):
                shuffled_connfile = os.path.join(metadatadir, randpat, 'tvb', 'connectivity.dk.zip')

            conn = connectivity.Connectivity.from_file(shuffled_connfile)
        elif patient is None and not other_pats:
            # shuffle within patients
            randweights = MainTVBSim().randshuffleweights(conn.weights)
            conn.weights = randweights
    return conn

def initialize_tvb_model(loader, ezregions, pzregions, period):
    ###################### INITIALIZE TVB SIMULATOR ##################
    maintvbexp = MainTVBSim(loader.conn, condspeed=np.inf)
    # load the necessary data files to run simulation
    maintvbexp.loadseegxyz(seegfile=loader.seegfile)
    maintvbexp.loadgainmat(gainfile=loader.gainfile)
    maintvbexp.importsurfdata(surf=loader.surf)

    ######### Model (Epileptor) Parameters ##########
    epileptor_params = {
        'r': 0.00037,#/1.5   # Temporal scaling in the third state variable
        'Ks': -10,                 # Permittivity coupling, fast to slow time scale
        'tt': 0.07,                   # time scale of simulation
        'tau': 10,                   # Temporal scaling coefficient in fifth st var
        'x0': -2.45, # x0c value = -2.05
        # 'Iext': iext,
    }
    x0ez=-1.65
    x0pz=-2.0 # x0pz = None
    if maintvbexp.ezregion is None:
        x0ez = None
    if maintvbexp.pzregion is None:
        x0pz = None
    maintvbexp.loadepileptor(ezregions=ezregions, pzregions=pzregions,
                            x0ez=x0ez, x0pz=x0pz,
                            epileptor_params=epileptor_params)
    allindices = np.hstack((maintvbexp.ezind, maintvbexp.pzind)).astype(int) 
    show_debug(maintvbexp)
    maintvbexp.ezindices = allindices
    ######### Integrator Parameters ##########
    n_tau = 0
    noise_cov = np.array([0.001, 0.001, 0.,\
                              0.0001, 0.0001, 0.])
    # define cov noise for the stochastic heun integrator
    hiss = noise.Additive(nsig=noise_cov, ntau=ntau)
    # hiss = noise.Multiplicative(nsig=noise_cov)
    integrator_params = {
        'dt': 0.05,
        'noise': hiss,
    }
    maintvbexp.loadintegrator(integrator_params)

    # load couping
    coupling_params = {
        'a': 1.,
    }
    maintvbexp.loadcoupling(**coupling_params)

    # load monitors
    initcond = None
    monitor_params = {
        'period': period,
        'moved': False,
        'initcond': initcond
    }
    maintvbexp.loadmonitors(**monitor_params)
    return maintvbexp

In [ ]:
# extract passed in variable
patient = 'id001_bt'
outputdatadir = '/Users/adam2392/Downloads/tvbexpsim/'
metadatadir = '/Users/adam2392/Downloads/tngpipeline/'
shuffleweights = True

# set all directories to output data, get meta data, get raw data
outputdatadir = os.path.join(outputdatadir, patient)
if not os.path.exists(outputdatadir):
    os.makedirs(outputdatadir)

# define the parameter sweeping by changing iext
iext_param_sweep = np.arange(2.0,4.0,0.1)
iext_param_sweep = [3.0]

# simulation parameters
_factor = 1
_samplerate = 1000*_factor # Hz
sim_length = 20*_samplerate    
period = 1./_factor

rawdatadir = os.path.join(metadatadir, patient)

# define sloader for this patient
loader = Subject(name=patient, root_pat_dir=rawdatadir, preload=False)

In [ ]:
## OUTPUTFILE NAME ##
filename = os.path.join(outputdatadir,
            '{0}_sim.npz'.format(patient))
metafilename = os.path.join(outputdatadir,
            '{0}_sim.json'.format(patient))
direc, simfilename = os.path.split(filename)

# get the ez/pz indices we want to use
clinezinds = loader.ezinds
clinpzinds = []
clinezregions = list(loader.conn['region_labels'][clinezinds])
clinpzregions = []

modelezinds = clinezinds
modelpzinds = clinpzinds
# allclinregions = clinezregions + clinpzregions
# sys.stdout.write("All clinical regions are: {}".format(allclinregions))

maintvbexp = initialize_tvb_model(loader, ezinds=modelezinds, pzinds=modelpzinds, period=period)
